In [26]:
import pandas as pd
import numpy as np
import pvlib
from pvlib.location import Location
import sympy as sm
from scipy import integrate

def inner(x1: sm.Matrix,x2: sm.Matrix):
    '''
    Computes the inner product of two vectors of same length.
    '''

    return x1.dot(x2, conjugate_convention = 'right')

sm.MutableDenseMatrix.inner = inner
sm.ImmutableDenseMatrix.inner = inner
L = 1.755
B = 1.038
maxFlux = 1000
maxEffect = 365/(L*B)
A = 0.5
S0 = 1100
S0A = S0 * A
effectPerFlux = maxEffect/maxFlux

In [27]:
def solar_elevation_angledeg(angle):
    return 90-angle

In [28]:
def solar_panel_projection(θ_s, ϕ_s, θ_p, ϕ_p):
    results = np.zeros(len(θ_s))
    for i in range(0,len(θ_s)):
        us = [-np.sin(θ_s[i]) * np.cos(ϕ_s[i]),-np.sin(θ_s[i]) * np.sin(ϕ_s[i]), -np.cos(θ_s[i])]
        up = [-1*np.sin(θ_p[i]) * np.cos(ϕ_p[i]), -np.sin(ϕ_p[i])*np.sin(θ_p[i]), -np.cos(θ_p[i])]
        inprod = (np.inner(us,up))
        results[i]=inprod
    results=results[results>0]
    return results



In [29]:

def kWhYear(zenithangs,azimuthangs, theta_p, phi_p):
    t= np.arange(len(zenithangs))
    indexOfInterest = t[zenithangs[t]>=0]
    indexOfInterest = indexOfInterest[zenithangs[indexOfInterest]<=np.pi/2]
    zenithangs=zenithangs[indexOfInterest]
    azimuthangs=azimuthangs[indexOfInterest]
    
    theta_list= np.full(len(zenithangs),theta_p)
    phi_list=np.full(len(zenithangs),phi_p)
    
    projList =  solar_panel_projection(zenithangs,azimuthangs,theta_list,phi_list)
    projList =  projList*effectPerFlux*S0A
    projList[projList>maxEffect]=maxEffect    
    return ((integrate.simpson(projList,dx = 3600))*L*B)/3600000

In [30]:
tidszone = "Europe/Copenhagen"
start_dato = "2024-01-01"
slut_dato = "2024-12-31"
delta_tid = "H" # "Min", "H",
# Definition of Location object. Coordinates and elevation of DTU, Copenhagen (Denmark)
site = Location(
55.7861111111, 12.5230555556, tidszone, 10, "DTU (DK)"
) # latitude, longitude, time_zone, altitude, name
# Definition of a time range of simulation
times = pd.date_range(
start_dato + " 00:00:00", slut_dato + " 23:59:00", inclusive="left", freq=delta_tid, tz=tidszone
)
# Estimate Solar Position with the ’Location’ object
solpos = site.get_solarposition(times)
results = []
#Lister af solens Zenith og Azimuth vinkler hver time over et  ̊ar
zenithangs = np.deg2rad(np.array(solpos.loc[start_dato:slut_dato].apparent_zenith))
azimuthangs = np.deg2rad(np.array(solpos.loc[start_dato:slut_dato].azimuth))
# Azimuth vinkel for panelet
phi_p = 180

#Der itereres over panelets zenith vinkler fra 0 til 90
for i in range(0,91):
    theta_p = i
    
 # Effekten for et helt  ̊ar ved en given zenith vinkel for panelet beregnes
    energyPrYear = kWhYear(zenithangs,azimuthangs, np.deg2rad(theta_p), np.deg2rad(phi_p))
    results.append(energyPrYear)
# Finder maks effekt og tilsvarende vinkel
max = 0
angle = -1
for i in range(0,len(results)):
    print(results[i])
    if(results[i] > max):
        max = results[i]
        angle = i
print("Energy Production is:",max,
"Angle is:",solar_elevation_angledeg(angle),"Degrees from horizon",)

337.82509064147666
343.9330808487314
349.9711957781455
356.01884514713254
362.0814168437266
367.713956531926
373.4153921310897
379.34448344831935
384.86261077504065
390.8097099572764
396.3456123325702
401.582389325118
407.0936468549035
412.03208601099635
417.1598671044049
422.0658863731435
427.09547167788566
431.6287771539241
436.4366250739413
440.6867779680704
444.9277317377383
449.1041798769369
454.08362293503274
458.0010628310881
461.2980987144679
465.64989369511517
468.9190056823618
472.9111639615116
475.8843683563681
479.33509444279645
482.20191675657384
485.46750479620795
488.1772319136651
491.08588669185076
493.6776700461101
495.9077237651038
498.41322365700717
500.2955762686786
502.2566087564733
504.05486789269486
506.14077311321427
507.1871837404174
508.5281117342876
510.4087423427366
511.5363891686873
512.5080410672047
513.3443006388034
513.1169686342614
513.607642476924
515.0018328794681
515.2865388563033
515.4743849494683
515.5818962388123
515.3885514407692
515.038214297276